Tools and Rounting 

LLM should decide which function to use and what the inputs to that function will be and calling the function with those inputs.
 Tools combine both of these ideas

So we can basically convert a function into a tool

Creating our own tools

Tool -  structured interaction with the external services

In [3]:
import os
import openai
from dotenv import load_dotenv, find_dotenv
_ = load_dotenv(find_dotenv()) # read local .env file
openai.api_key = os.environ['OPENAI_API_KEY']

In [4]:
from langchain.agents import tool

In [5]:
@tool
def search(query:str)->str: #takes a string and indicates that the function is expected to return a string
    """Search for the wether online"""
    return "42f"

In [6]:
search.name

'search'

In [7]:
search

StructuredTool(name='search', description='Search for the wether online', args_schema=<class 'langchain_core.utils.pydantic.search'>, func=<function search at 0x08F24F10>)

In [8]:
search.args

{'query': {'title': 'Query', 'type': 'string'}}

We can improve upon this by defining a more explicit structure by the input schema

In [9]:
from pydantic import BaseModel, Field
class SearchInput(BaseModel):
    query:str = Field(description="thing to search for")

In [10]:
@tool(args_schema=SearchInput)
def search(query:str)->str: #takes a string and indicates that the function is expected to return a string
    """Search for the wether online"""
    return "42f"

In [11]:
search.run("dl")

'42f'

So we are creating a tool that will give the temperature given a latitude and the longitude

In [12]:
import requests
from pydantic import BaseModel, Field
import datetime

In [13]:
class OpenMeteo(BaseModel):
    latitude : float = Field(description="latitude of the location")
    longitude: float = Field(description="longitude of the location")

In [23]:
@tool(args_schema=OpenMeteo)
def get_current_temperature(latitude:float,longitude :float)->dict:
    """Get the current temperature of the given coordinates"""
    BASE_URL = "https://api.open-meteo.com/v1/forecast"

    params = {
        'latitude':latitude,
        'longitude':longitude,
        'hourly': 'temperature_2m',
        'forecast_days': 1,
    }

    #Make the resuest
    response = requests.get(BASE_URL, params=params)

    if response.status_code==200: #status code if the resquest was successful
        return response.json()
    else:
        raise Exception(f"API request failes with the status code :{response.status_code}")
    
    current_utc_time = datetime.datetime.utcnow()
    time_list = [datetime.datetime.fromisoformat(time_str.replace('Z', '+00:00')) for time_str in results['hourly']['time']]
    temperature_list = results['hourly']['temperature_2m']
    
    closest_time_index = min(range(len(time_list)), key=lambda i: abs(time_list[i] - current_utc_time))
    current_temperature = temperature_list[closest_time_index]
    
    return f'The current temperature is {current_temperature}°C'

In [24]:
get_current_temperature.name

'get_current_temperature'

In [25]:
get_current_temperature.description

'Get the current temperature of the given coordinates'

In [26]:
get_current_temperature.args

{'latitude': {'description': 'latitude of the location',
  'title': 'Latitude',
  'type': 'number'},
 'longitude': {'description': 'longitude of the location',
  'title': 'Longitude',
  'type': 'number'}}

Now we can covert this tool to the openai function

In [27]:
from langchain.tools.render import format_tool_to_openai_function

In [28]:
format_tool_to_openai_function(get_current_temperature)

{'name': 'get_current_temperature',
 'description': 'Get the current temperature of the given coordinates',
 'parameters': {'properties': {'latitude': {'description': 'latitude of the location',
    'type': 'number'},
   'longitude': {'description': 'longitude of the location',
    'type': 'number'}},
  'required': ['latitude', 'longitude'],
  'type': 'object'}}

In [29]:
get_current_temperature({"latitude":28.737651005980887,"longitude" : 77.12872502495725})

{'latitude': 28.75,
 'longitude': 77.125,
 'generationtime_ms': 0.0133514404296875,
 'utc_offset_seconds': 0,
 'timezone': 'GMT',
 'timezone_abbreviation': 'GMT',
 'elevation': 224.0,
 'hourly_units': {'time': 'iso8601', 'temperature_2m': '°C'},
 'hourly': {'time': ['2025-01-16T00:00',
   '2025-01-16T01:00',
   '2025-01-16T02:00',
   '2025-01-16T03:00',
   '2025-01-16T04:00',
   '2025-01-16T05:00',
   '2025-01-16T06:00',
   '2025-01-16T07:00',
   '2025-01-16T08:00',
   '2025-01-16T09:00',
   '2025-01-16T10:00',
   '2025-01-16T11:00',
   '2025-01-16T12:00',
   '2025-01-16T13:00',
   '2025-01-16T14:00',
   '2025-01-16T15:00',
   '2025-01-16T16:00',
   '2025-01-16T17:00',
   '2025-01-16T18:00',
   '2025-01-16T19:00',
   '2025-01-16T20:00',
   '2025-01-16T21:00',
   '2025-01-16T22:00',
   '2025-01-16T23:00'],
  'temperature_2m': [11.6,
   11.7,
   11.7,
   12.3,
   13.7,
   15.3,
   15.3,
   16.6,
   17.2,
   17.6,
   17.3,
   16.8,
   16.2,
   15.1,
   14.2,
   13.4,
   12.7,
   12.1,
   

Second tool we are going to make is a wikipedia tool which is going to search things in wikipedia

In [32]:
import wikipedia
@tool
def search_wikipedia(query:str)->str:
    """Run wikipedia search and get the page summaries"""
    page_titles = wikipedia.search(query)
    summaries = []
    for page_title in page_titles[: 3]:
        try:
            wiki_page =  wikipedia.page(title=page_title, auto_suggest=False)
            summaries.append(f"Page: {page_title}\nSummary: {wiki_page.summary}")
        except (
            self.wiki_client.exceptions.PageError,
            self.wiki_client.exceptions.DisambiguationError,
        ):
            pass
    if not summaries:
        return "No good Wikipedia Search Result was found"
    return "\n\n".join(summaries)

In [34]:
search_wikipedia.name

'search_wikipedia'

In [35]:
search_wikipedia.description

'Run wikipedia search and get the page summaries'

In [36]:
format_tool_to_openai_function(search_wikipedia)

{'name': 'search_wikipedia',
 'description': 'Run wikipedia search and get the page summaries',
 'parameters': {'properties': {'query': {'type': 'string'}},
  'required': ['query'],
  'type': 'object'}}

In [37]:
search_wikipedia({"query":"tyagi"})

"Page: Tyagi\nSummary: Tyagi, originally called Taga, is a cultivator caste who claim Brahmin status. The landholding community is confined to Western Uttar Pradesh, Haryana, Delhi and Rajasthan. They are often considered the highest of the agricultural castes. During the British Raj, they changed their name from Taga to Tyagi, and began claiming Brahmin status. As of a 1990 report by the Backward Classes Commission, Government of Haryana, they were mostly engaged in farming. The Government of Haryana granted reservation to Tyagis along with five other castes in 2016. However, the Punjab and Haryana High Court shortly put a stay on the government's order.\n\n\n\nPage: Sachin Tyagi\nSummary: Sachin Tyagi is an Indian Hindi television actor and singer, best known for playing Manish Goenka in the TV soap Yeh Rishta Kya Kehlata Hai starting from 2016 . He is married to actress Rakshanda Khan.\n\nPage: Sucharita Tyagi\nSummary: Sucharita Tyagi is an Indian film critic and former radio jocke

Routing 
Deciding the Language model which path to take and also the input to that path.
Basically if we have 2 functions then which function the language model should choose and the inputs to that function

In [38]:
from langchain.chat_models import ChatOpenAI

In lesson3, we show an example of function calling deciding between 2 candidate functions.
Given our tools above we can format these as openai functions and show this same behavouir

In [40]:
functions = [
    format_tool_to_openai_function(f) for f in [ search_wikipedia, get_current_temperature]
]
model = ChatOpenAI(temperature = 0).bind(functions = functions)

C:\Users\HP\AppData\Local\Temp\ipykernel_10124\2191828343.py:4: LangChainDeprecationWarning: The class `ChatOpenAI` was deprecated in LangChain 0.0.10 and will be removed in 1.0. An updated version of the class exists in the :class:`~langchain-openai package and should be used instead. To use it run `pip install -U :class:`~langchain-openai` and import as `from :class:`~langchain_openai import ChatOpenAI``.
  model = ChatOpenAI(temperature = 0).bind(functions = functions)


In [41]:
model.invoke("what is the weather like in DL now ?")

AIMessage(content='', additional_kwargs={'function_call': {'arguments': '{"latitude":28.7041,"longitude":77.1025}', 'name': 'get_current_temperature'}}, response_metadata={'token_usage': {'completion_tokens': 26, 'prompt_tokens': 99, 'total_tokens': 125, 'completion_tokens_details': {'accepted_prediction_tokens': 0, 'audio_tokens': 0, 'reasoning_tokens': 0, 'rejected_prediction_tokens': 0}, 'prompt_tokens_details': {'audio_tokens': 0, 'cached_tokens': 0}}, 'model_name': 'gpt-3.5-turbo', 'system_fingerprint': None, 'finish_reason': 'function_call', 'logprobs': None}, id='run-e6922cff-4fef-4870-abc5-643bfb1d56a7-0')

In [42]:
model.invoke("WHat is langchain")

AIMessage(content='', additional_kwargs={'function_call': {'arguments': '{"query":"Langchain"}', 'name': 'search_wikipedia'}}, response_metadata={'token_usage': {'completion_tokens': 17, 'prompt_tokens': 95, 'total_tokens': 112, 'completion_tokens_details': {'accepted_prediction_tokens': 0, 'audio_tokens': 0, 'reasoning_tokens': 0, 'rejected_prediction_tokens': 0}, 'prompt_tokens_details': {'audio_tokens': 0, 'cached_tokens': 0}}, 'model_name': 'gpt-3.5-turbo', 'system_fingerprint': None, 'finish_reason': 'function_call', 'logprobs': None}, id='run-05e86c28-64c0-473c-8872-0af6fabf7e2f-0')

In [53]:
from langchain.prompts import ChatPromptTemplate
from langchain.agents.output_parsers import OpenAIFunctionsAgentOutputParser

In [54]:
prompt = ChatPromptTemplate.from_messages(
    [
        ("system","you are helpful but sassy assistant"),
        ("user","{input}"),
    ]
)

In [55]:
chain = prompt | model | OpenAIFunctionsAgentOutputParser()

In [56]:
chain.invoke("Whats the weather like in delhi right now")

AgentActionMessageLog(tool='get_current_temperature', tool_input={'latitude': 28.7041, 'longitude': 77.1025}, log="\nInvoking: `get_current_temperature` with `{'latitude': 28.7041, 'longitude': 77.1025}`\n\n\n", message_log=[AIMessage(content='', additional_kwargs={'function_call': {'arguments': '{"latitude":28.7041,"longitude":77.1025}', 'name': 'get_current_temperature'}}, response_metadata={'token_usage': {'completion_tokens': 26, 'prompt_tokens': 107, 'total_tokens': 133, 'completion_tokens_details': {'accepted_prediction_tokens': 0, 'audio_tokens': 0, 'reasoning_tokens': 0, 'rejected_prediction_tokens': 0}, 'prompt_tokens_details': {'audio_tokens': 0, 'cached_tokens': 0}}, 'model_name': 'gpt-3.5-turbo', 'system_fingerprint': None, 'finish_reason': 'function_call', 'logprobs': None}, id='run-8dca5d27-30c3-45ef-bab7-1ad046762cfe-0')])